## 5.보험 예측

In [28]:
!git clone https://github.com/Soyoung-Yoon/bigdata

fatal: destination path 'bigdata' already exists and is not an empty directory.


### 5-1. 데이터 이해, 생성
- 1개 데이터를 나누어 시험용으로 변경해 보는 작업
- 실제 시험에서는 이런 과정은 필요 없음


In [29]:
# https://www.kaggle.com/qusaybtoush/insurance-forecast-by-using-linear-regression
# Age: age of primary beneficiary
# Sex: insurance contractor gender, female, male
# bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height, objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9
# Children: Number of children covered by health insurance / Number of dependents
# Smoker: Smoking
# Region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.
# Charges: Individual medical costs billed by health insurance

컬럼 정보
- age : 1차 수혜자 연령
- sex : 보험 계약자 성별
- bmi : 비만도 지수
- children : 건강보험 적용 대상 아동 수
- smoker : 흡연여부
- region : 거주지역
- charges : 의료 보험에서 청구하는 개별 의료비

In [30]:
# [0] 사용 라이브러리 import
import pandas as pd

# 데이타가 많은경우 모두 출력 안되고 ... 으로 생략해서 출력됨.
# 시험환경에서는 아래와 같이 해야해서 수정했습니다 ^^*
pd.options.display.max_rows = 500     #출력할 max row를 지정
pd.options.display.max_columns = 20   #출력할 max columns를 지정
#출력 format 지정 - 소수점아래 4자리까지
pd.set_option('display.float_format', '{:.4f}'.format)

In [31]:
# [1] 'bigdata/insurance.csv' 파일 불러오기, shape 확인하기
df = pd.read_csv('bigdata/insurance.csv')
df.shape

(1338, 7)

In [32]:
# [2] head를 사용해 데이터의 모습 확인  (Y => charges)
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.9000,0,yes,southwest,16884.9240
1,18,male,33.7700,1,no,southeast,1725.5523
2,28,male,33.0000,3,no,southeast,4449.4620
3,33,male,22.7050,0,no,northwest,21984.4706
4,32,male,28.8800,0,no,northwest,3866.8552


In [33]:
# [3] 시험 형식으로 train, test 데이터로 나누어 저장합니다.  (6:4 비율)
train_size = len(df) - int(len(df)*0.4,)

# 여러 가지 데이터가 고루 섞이도록 해야 함
df = df.sample(frac=1,random_state=1234)
df.insert(0,'ID',range(1,len(df)+1))
train = df.iloc[:train_size, ]
test = df.iloc[train_size:, ]
print(train.shape, test.shape)
ycolumn = 'charges'
X_train = train.drop(columns=ycolumn)
Y_train = train[['ID', ycolumn]]
X_test = test.drop(columns=ycolumn)
Y_test = test[['ID', ycolumn]]
X_train.to_csv('x_train.csv', index=False)
Y_train.to_csv('y_train.csv', index=False)
X_test.to_csv('x_test.csv', index=False)
Y_test.to_csv('y_test.csv', index=False)

(803, 8) (535, 8)


In [34]:
display(X_train.head(),Y_train.head())

,ID,age,sex,bmi,children,smoker,region
395,1,46,male,19.8550,0,no,northwest
809,2,25,male,25.8400,1,no,northeast
324,3,29,male,27.2000,0,no,southwest
237,4,31,male,38.3900,2,no,southeast
1132,5,57,male,40.2800,0,no,northeast


,ID,charges
395,1,7526.7064
809,2,3309.7926
324,3,2866.0910
237,4,4463.2051
1132,5,20709.0203


### 5-2. 데이터 불러오기, 전처리

In [35]:
# [1] 데이터 파일 불러오기
X_use = pd.read_csv('x_train.csv')
X_submission = pd.read_csv('x_test.csv')
Y = pd.read_csv('y_train.csv')
Y_hidden = pd.read_csv('y_test.csv')

In [36]:
# [2] dfX 만들기, dfX의 info() 확인
# use, submission 데이터를 묶어 전처리 하기 위함
dfX = pd.concat([X_use,X_submission])
dfX.info()
dfX.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 1338 entries, 0 to 534
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        1338 non-null   int64  
 1   age       1338 non-null   int64  
 2   sex       1338 non-null   object 
 3   bmi       1338 non-null   float64
 4   children  1338 non-null   int64  
 5   smoker    1338 non-null   object 
 6   region    1338 non-null   object 
dtypes: float64(1), int64(3), object(3)
memory usage: 83.6+ KB


,ID,age,bmi,children
count,1338.0000,1338.0000,1338.0000,1338.0000
mean,669.5000,39.2070,30.6634,1.0949
std,386.3916,14.0500,6.0982,1.2055
min,1.0000,18.0000,15.9600,0.0000
25%,335.2500,27.0000,26.2963,0.0000
50%,669.5000,39.0000,30.4000,1.0000
75%,1003.7500,51.0000,34.6938,2.0000
max,1338.0000,64.0000,53.1300,5.0000


In [37]:
dfX.isna().sum().sum()

0

In [38]:
# [3] 컬럼별 고윳값 데이터 수 개수 확인
dfX.nunique()

,0
ID,1338
age,47
sex,2
bmi,548
children,6
smoker,2
region,4


In [39]:
# [4] object 컬럼들에 대해서 Encoding 실행 - 필수
for x in ['sex','smoker','region']:
  temp = dfX[x].unique()
  dfX[x] = dfX[x].replace(temp,range(len(temp)))

<ipython-input-39-48929b7c951e>:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfX[x] = dfX[x].replace(temp,range(len(temp)))


In [40]:
dfX.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1338 entries, 0 to 534
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        1338 non-null   int64  
 1   age       1338 non-null   int64  
 2   sex       1338 non-null   int64  
 3   bmi       1338 non-null   float64
 4   children  1338 non-null   int64  
 5   smoker    1338 non-null   int64  
 6   region    1338 non-null   int64  
dtypes: float64(1), int64(6)
memory usage: 83.6 KB


In [41]:
# [5] dfX와 Y를 합하여 dfXY 생성
dfXY = pd.merge(dfX,Y)

In [42]:
# [6] 'charges'와 다른 컬럼의 상관계수를 확인합니다.
dfXY.corr()['charges'].abs().sort_values(ascending=False)

,charges
charges,1.0000
smoker,0.7829
age,0.3158
bmi,0.2112
region,0.0594
sex,0.0546
children,0.0512
ID,0.0166


### 5-3. 성능평가, 모델링 함수


In [43]:
# 사용할 라이브러리 import
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso,Ridge

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_log_error as msle
from sklearn.metrics import r2_score


In [44]:
# r2_train, r2_test, mae, mse, rmse, msle, rmsle 를 구해
# Series로 반환하는 함수를 작성합니다.

def get_scores(model,xtrain,xtest,ytrain,ytest):
  pred1 = model.predict(xtrain) # train
  pred2 = model.predict(xtest)  # test

  #음수가 있어도 상관없음
  A1 = r2_score(ytrain,pred1)
  A2 = r2_score(ytest,pred2)
  B = mae(ytest,pred2)
  C = mse(ytest,pred2)
  E = np.sqrt(C) # rmse

  #음수가 있으면 안됨
  pred2 = np.where(pred2<0,0,pred2)
  D = msle(ytest,pred2)
  F = np.sqrt(D) # rmsle

  data = [round(x,4) for x in [A1,A2,B,C,D,E,F]]
  names = 'r2_train r2_test mae mse msle rmse rmsle'.split()
  scores = pd.Series(data,index=names)
  return scores

In [45]:
# 다양한 모델을 만들고 성능을 출력하는 함수 작성
# 결과를 DataFrame으로 만듭니다

def make_models(xtrain,xtest,ytrain,ytest,n=300,RL=False):
  temp = pd.DataFrame()

  model1 = LinearRegression().fit(xtrain,ytrain)
  temp['model1'] = get_scores(model1,xtrain,xtest,ytrain,ytest)

  if not RL:
    model2 = DecisionTreeRegressor(random_state=0).fit(xtrain,ytrain)
    temp['model2'] = get_scores(model2,xtrain,xtest,ytrain,ytest)

    for d in range(3,9):
      model2 = DecisionTreeRegressor(max_depth=d,random_state=0).fit(xtrain,ytrain)
      temp[f'model2_{d}'] = get_scores(model2,xtrain,xtest,ytrain,ytest)

    model3 = RandomForestRegressor(n,random_state=0).fit(xtrain,ytrain)
    temp['model3'] = get_scores(model3,xtrain,xtest,ytrain,ytest)

    for d in range(3,9):
      model3= RandomForestRegressor(n,max_depth=d,random_state=0).fit(xtrain,ytrain)
      temp[f'model3_{d}'] = get_scores(model3,xtrain,xtest,ytrain,ytest)

  model4 = XGBRegressor(objective = 'reg:squarederror').fit(xtrain,ytrain)
  temp['model4'] = get_scores(model4,xtrain,xtest,ytrain,ytest)

  if RL:
    for a in [0.01, 0.1, 1, 2]:
      model5 = Ridge(alpha=a).fit(xtrain,ytrain)
      temp['model5'] = get_scores(model5,xtrain,xtest,ytrain,ytest)

    for a in [0.01, 0.1, 1, 2]:
      model6 = Lasso(alpha=a).fit(xtrain,ytrain)
      temp['model6'] = get_scores(model6,xtrain,xtest,ytrain,ytest)

  temp = temp.T
  temp.insert(2,'diff',(temp['r2_train']-temp['r2_test']).abs())

  return temp


### 5-4. 데이터 분리, 모델적용

In [46]:
# [7] X의 최종 전처리 버전인 dfX의 컬럼 확인
dfX.columns

Index(['ID', 'age', 'sex', 'bmi', 'children', 'smoker', 'region'], dtype='object')

In [47]:
# [8] dfX 에서 사용할 feature 선택하여 dfX2로 저장
dfX2 = dfX[['age', 'sex', 'bmi', 'children', 'smoker', 'region']]
dfX2

,age,sex,bmi,children,smoker,region
0,46,0,19.8550,0,0,0
1,25,0,25.8400,1,0,1
2,29,0,27.2000,0,0,2
3,31,0,38.3900,2,0,3
4,57,0,40.2800,0,0,1
...,...,...,...,...,...,...
530,56,0,34.4300,0,0,3
531,21,0,26.0300,0,0,1
532,35,0,39.7100,4,0,1
533,19,0,35.4000,0,0,2


In [48]:
# [9] dfX2 데이터를 XF, X_submissionF 로 다시 분리
# dfX2 분리에서는 X_use의 행의 개수 사용
# Y에서 'charges' 만 YF로 사용
train_size = len(X_use)
XF = dfX2[:train_size]
X_submissionF = dfX2[train_size:]
print(len(XF),len(X_submission))

YF = Y['charges']
Y_submissionF = Y_hidden['charges']
print(len(YF),len(Y_submissionF))


803 535
803 535


In [49]:
# [10] XF, YF 데이터를 사용하여 데이터를 분리하고 make_models를 호출합니다.
xtrain,xtest,ytrain,ytest = train_test_split(XF,YF,
                                             test_size=0.3,
                                             random_state=1234)
models = make_models(xtrain,xtest,ytrain,ytest)
print(models)

          r2_train  r2_test   diff       mae           mse   msle      rmse  \
model1      0.7276   0.8053 0.0777 4088.1861 34101788.3022 2.2322 5839.6736   
model2      1.0000   0.7819 0.2181 2868.9598 38205213.0025 0.1835 6181.0366   
model2_3    0.8585   0.8858 0.0273 2696.0927 19997881.3735 0.1821 4471.8991   
model2_4    0.8727   0.8958 0.0231 2577.9273 18253288.2026 0.1451 4272.3867   
model2_5    0.8874   0.8728 0.0146 2596.6833 22286045.4088 0.1322 4720.8098   
model2_6    0.9099   0.8579 0.0520 2577.6188 24890079.4374 0.1345 4988.9958   
model2_7    0.9336   0.8346 0.0990 2798.3615 28976483.8421 0.1562 5382.9809   
model2_8    0.9529   0.8257 0.1272 2790.4426 30532221.1069 0.1581 5525.5969   
model3      0.9745   0.8859 0.0886 2463.1906 19989477.3933 0.1441 4470.9593   
model3_3    0.8655   0.8890 0.0235 2595.3486 19453820.2124 0.1677 4410.6485   
model3_4    0.8802   0.8934 0.0132 2530.0964 18675883.3582 0.1363 4321.5603   
model3_5    0.8999   0.8920 0.0079 2456.3668 1892692

In [50]:
# [11] models를 평가기준으로 정렬하여 좋은 모델을 알아봄
print(models.sort_values('rmsle').head(5))

          r2_train  r2_test   diff       mae           mse   msle      rmse  \
model3_6    0.9229   0.8915 0.0314 2412.9721 19010932.4231 0.1275 4360.1528   
model3_5    0.8999   0.8920 0.0079 2456.3668 18926929.1843 0.1285 4350.5091   
model3_7    0.9443   0.8900 0.0543 2415.4275 19264118.1168 0.1312 4389.0908   
model2_5    0.8874   0.8728 0.0146 2596.6833 22286045.4088 0.1322 4720.8098   
model2_6    0.9099   0.8579 0.0520 2577.6188 24890079.4374 0.1345 4988.9958   

          rmsle  
model3_6 0.3570  
model3_5 0.3584  
model3_7 0.3622  
model2_5 0.3636  
model2_6 0.3668  


### 5-5. 모델 선택, 결과 제출

In [51]:
# [12] 모델 학습시켜 model 로 저장, submission 만들기, 저장하기
model = RandomForestRegressor(300,max_depth=5,random_state=0).fit(xtrain,ytrain)
print(get_scores(model,xtrain,xtest,ytrain,ytest))

pred = model.predict(X_submissionF)
submission = pd.DataFrame({'ID': X_submission['ID'],
                           'charges': pred})
submission.to_csv('00100.csv',index=False)

r2_train          0.8999
r2_test           0.8920
mae            2456.3668
mse        18926929.1843
msle              0.1285
rmse           4350.5091
rmsle             0.3584
dtype: float64


In [52]:
# [13] Y_submissionF 를 사용하여 성능을 알아봄 (필수 아님)
temp = pd.DataFrame({'ID':X_submission['ID'],
              'charges_pred' : pred,
              'charges_true' : Y_submissionF})
temp.head(10)

,ID,charges_pred,charges_true
0,804,3305.5344,1163.4627
1,805,39382.0442,38792.6856
2,806,8380.3883,18838.7037
3,807,10700.0797,8062.7640
4,808,14171.3555,12979.3580
5,809,13555.1580,30063.5805
6,810,5042.0867,3577.9990
7,811,7279.0083,7727.2532
8,812,4417.3051,2352.9684
9,813,17193.4562,13063.8830


### 5-6.시험보러 가서는 이렇게

In [53]:
# 실제 작업

# import 를 쭈우욱!! 작성
import pandas as pd
# 시험환경에서는 아래와 같이 해야해서 수정했습니다 ^^*
pd.options.display.max_rows = 500     #출력할 max row를 지정
pd.options.display.max_columns = 20   #출력할 max columns를 지정
pd.set_option('display.float_format', '{:.4f}'.format)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_squared_log_error as msle
import numpy as np  # 추가해야함

# 함수들은 만들어야 합니다.

def get_scores_f(model, xtrain, xtest, ytrain, ytest):
    pred1 = model.predict(xtrain)   # train
    pred2 = model.predict(xtest)    # test

    # 음수가 있어도 상관없음
    A1 = r2_score(ytrain, pred1)
    A2 = r2_score(ytest, pred2)
    # msle, rmsle 는 음수가 있어서는 안됨
    pred2 = np.where(pred2<0, 0, pred2)
    B = msle(ytest, pred2)
    C = np.sqrt(B)   # rmsle
    data = [round(x, 4) for x in [A1, A2, C]]
    names = 'r2_train r2_test rmsle'.split()
    scores = pd.Series(data, index=names)
    return  scores

def make_models_f(xtrain, xtest, ytrain, ytest, n=300):
    temp = pd.DataFrame()

    model1 = LinearRegression().fit(xtrain, ytrain)
    temp['model1'] = get_scores_f(model1, xtrain, xtest, ytrain, ytest)

    model2 = DecisionTreeRegressor(random_state=0).fit(xtrain, ytrain)  # overfitting 나기 쉬움
    temp['model2'] = get_scores_f(model2, xtrain, xtest, ytrain, ytest)

    for d in range(3, 9):
        model2 = DecisionTreeRegressor(max_depth=d, random_state=0).fit(xtrain, ytrain)
        temp[f'model2_{d}'] = get_scores_f(model2, xtrain, xtest, ytrain, ytest)

    model3 = RandomForestRegressor(n, random_state=0).fit(xtrain, ytrain)
    temp['model3'] = get_scores_f(model3, xtrain, xtest, ytrain, ytest)

    for d in range(3, 9):
        model3 = RandomForestRegressor(n, max_depth=d, random_state=0).fit(xtrain, ytrain)
        temp[f'model3_{d}'] = get_scores_f(model3, xtrain, xtest, ytrain, ytest)

    model4 = XGBRegressor(objective='reg:squarederror').fit(xtrain, ytrain)
    temp['model4'] = get_scores_f(model4, xtrain, xtest, ytrain, ytest)

    temp = temp.T
    temp.insert(2, 'diff', (temp['r2_train'] - temp['r2_test']).abs())

    return temp

X_use = pd.read_csv('x_train.csv')
X_submission = pd.read_csv('x_test.csv')
Y = pd.read_csv('y_train.csv')
#print([x.shape for x in [X_use, X_submission, Y]]) # [(803, 7), (535, 7), (803, 2)]
# ----------------------------------------------------------------------------------
dfX = pd.concat([X_use, X_submission], ignore_index=True)
#dfX.info()
#print(dfX.nunique())
#print(dfX['region'].unique())

for x in ['sex', 'smoker', 'region']:
    temp = dfX[x].unique()
    dfX[x] = dfX[x].replace(temp, range(len(temp)))
#dfX.info()

#print(dfX.columns)
dfX2 = dfX.drop(columns='ID')
#dfX2.columns

train_size = len(X_use)
XF = dfX2[:train_size]  #  dfX2가 DataFrame, ndarry 인 경우도 행번호로 indexing 가능
X_submissionF = dfX2[train_size:]
YF = Y['charges']
#print([x.shape for x in [XF, YF, X_submissionF]])

xtrain, xtest, ytrain, ytest = train_test_split(XF, YF,
                                                test_size=0.3,
                                                random_state=1234)


#models = make_models_f(xtrain, xtest, ytrain, ytest)
#print(models.sort_values('rmsle').head(5))
model = RandomForestRegressor(300, max_depth=5, random_state=0).fit(xtrain, ytrain)
print(get_scores_f(model, xtrain, xtest, ytrain, ytest))
pred = model.predict(X_submissionF)
submission = pd.DataFrame({'ID': X_submission['ID'],
                           'charges' : pred})
submission.to_csv('0001000.csv', index=False)

<ipython-input-53-21ee064f2585>:79: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfX[x] = dfX[x].replace(temp, range(len(temp)))


r2_train   0.8999
r2_test    0.8920
rmsle      0.3584
dtype: float64


In [54]:
# model3_5    0.8999   0.8920 0.0079 0.3584

## 5.보험 예측

In [55]:
!git clone https://github.com/Soyoung-Yoon/bigdata

fatal: destination path 'bigdata' already exists and is not an empty directory.


### 5-1. 데이터 이해, 생성
- 1개 데이터를 나누어 시험용으로 변경해 보는 작업
- 실제 시험에서는 이런 과정은 필요 없음


In [56]:
# https://www.kaggle.com/qusaybtoush/insurance-forecast-by-using-linear-regression
# Age: age of primary beneficiary
# Sex: insurance contractor gender, female, male
# bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height, objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9
# Children: Number of children covered by health insurance / Number of dependents
# Smoker: Smoking
# Region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.
# Charges: Individual medical costs billed by health insurance

컬럼 정보
- age : 1차 수혜자 연령
- sex : 보험 계약자 성별
- bmi : 비만도 지수
- children : 건강보험 적용 대상 아동 수
- smoker : 흡연여부
- region : 거주지역
- charges : 의료 보험에서 청구하는 개별 의료비

In [57]:
# [0] 사용 라이브러리 import
import pandas as pd

# 데이타가 많은경우 모두 출력 안되고 ... 으로 생략해서 출력됨.
# 시험환경에서는 아래와 같이 해야해서 수정했습니다 ^^*
pd.options.display.max_rows = 500     #출력할 max row를 지정
pd.options.display.max_columns = 20   #출력할 max columns를 지정
#출력 format 지정 - 소수점아래 4자리까지
pd.set_option('display.float_format', '{:.4f}'.format)

In [58]:
# [1] 'bigdata/insurance.csv' 파일 불러오기, shape 확인하기
df = pd.read_csv('bigdata/insurance.csv')
df.shape

(1338, 7)

In [59]:
# [2] head를 사용해 데이터의 모습 확인  (Y => charges)
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.9000,0,yes,southwest,16884.9240
1,18,male,33.7700,1,no,southeast,1725.5523
2,28,male,33.0000,3,no,southeast,4449.4620
3,33,male,22.7050,0,no,northwest,21984.4706
4,32,male,28.8800,0,no,northwest,3866.8552


In [60]:
# [3] 시험 형식으로 train, test 데이터로 나누어 저장합니다.  (6:4 비율)
train_size = len(df) - int(len(df)*0.4,)

# 여러 가지 데이터가 고루 섞이도록 해야 함
df = df.sample(frac=1,random_state=1234)
df.insert(0,'ID',range(1,len(df)+1))
train = df.iloc[:train_size, ]
test = df.iloc[train_size:, ]
print(train.shape, test.shape)
ycolumn = 'charges'
X_train = train.drop(columns=ycolumn)
Y_train = train[['ID', ycolumn]]
X_test = test.drop(columns=ycolumn)
Y_test = test[['ID', ycolumn]]
X_train.to_csv('x_train.csv', index=False)
Y_train.to_csv('y_train.csv', index=False)
X_test.to_csv('x_test.csv', index=False)
Y_test.to_csv('y_test.csv', index=False)

(803, 8) (535, 8)


In [61]:
display(X_train.head(),Y_train.head())

,ID,age,sex,bmi,children,smoker,region
395,1,46,male,19.8550,0,no,northwest
809,2,25,male,25.8400,1,no,northeast
324,3,29,male,27.2000,0,no,southwest
237,4,31,male,38.3900,2,no,southeast
1132,5,57,male,40.2800,0,no,northeast


,ID,charges
395,1,7526.7064
809,2,3309.7926
324,3,2866.0910
237,4,4463.2051
1132,5,20709.0203


### 5-2. 데이터 불러오기, 전처리

In [62]:
# [1] 데이터 파일 불러오기
X_use = pd.read_csv('x_train.csv')
X_submission = pd.read_csv('x_test.csv')
Y = pd.read_csv('y_train.csv')
Y_hidden = pd.read_csv('y_test.csv')

In [63]:
# [2] dfX 만들기, dfX의 info() 확인
# use, submission 데이터를 묶어 전처리 하기 위함
dfX = pd.concat([X_use,X_submission])
dfX.info()
dfX.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 1338 entries, 0 to 534
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        1338 non-null   int64  
 1   age       1338 non-null   int64  
 2   sex       1338 non-null   object 
 3   bmi       1338 non-null   float64
 4   children  1338 non-null   int64  
 5   smoker    1338 non-null   object 
 6   region    1338 non-null   object 
dtypes: float64(1), int64(3), object(3)
memory usage: 83.6+ KB


,ID,age,bmi,children
count,1338.0000,1338.0000,1338.0000,1338.0000
mean,669.5000,39.2070,30.6634,1.0949
std,386.3916,14.0500,6.0982,1.2055
min,1.0000,18.0000,15.9600,0.0000
25%,335.2500,27.0000,26.2963,0.0000
50%,669.5000,39.0000,30.4000,1.0000
75%,1003.7500,51.0000,34.6938,2.0000
max,1338.0000,64.0000,53.1300,5.0000


In [64]:
dfX.isna().sum().sum()

0

In [65]:
# [3] 컬럼별 고윳값 데이터 수 개수 확인
dfX.nunique()

,0
ID,1338
age,47
sex,2
bmi,548
children,6
smoker,2
region,4


In [66]:
# [4] object 컬럼들에 대해서 Encoding 실행 - 필수
for x in ['sex','smoker','region']:
  temp = dfX[x].unique()
  dfX[x] = dfX[x].replace(temp,range(len(temp)))

<ipython-input-66-48929b7c951e>:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfX[x] = dfX[x].replace(temp,range(len(temp)))


In [67]:
dfX.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1338 entries, 0 to 534
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        1338 non-null   int64  
 1   age       1338 non-null   int64  
 2   sex       1338 non-null   int64  
 3   bmi       1338 non-null   float64
 4   children  1338 non-null   int64  
 5   smoker    1338 non-null   int64  
 6   region    1338 non-null   int64  
dtypes: float64(1), int64(6)
memory usage: 83.6 KB


In [68]:
# [5] dfX와 Y를 합하여 dfXY 생성
dfXY = pd.merge(dfX,Y)

In [69]:
# [6] 'charges'와 다른 컬럼의 상관계수를 확인합니다.
dfXY.corr()['charges'].abs().sort_values(ascending=False)

,charges
charges,1.0000
smoker,0.7829
age,0.3158
bmi,0.2112
region,0.0594
sex,0.0546
children,0.0512
ID,0.0166


### 5-3. 성능평가, 모델링 함수


In [70]:
# 사용할 라이브러리 import
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso,Ridge

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_log_error as msle
from sklearn.metrics import r2_score


In [71]:
# r2_train, r2_test, mae, mse, rmse, msle, rmsle 를 구해
# Series로 반환하는 함수를 작성합니다.

def get_scores(model,xtrain,xtest,ytrain,ytest):
  pred1 = model.predict(xtrain) # train
  pred2 = model.predict(xtest)  # test

  #음수가 있어도 상관없음
  A1 = r2_score(ytrain,pred1)
  A2 = r2_score(ytest,pred2)
  B = mae(ytest,pred2)
  C = mse(ytest,pred2)
  E = np.sqrt(C) # rmse

  #음수가 있으면 안됨
  pred2 = np.where(pred2<0,0,pred2)
  D = msle(ytest,pred2)
  F = np.sqrt(D) # rmsle

  data = [round(x,4) for x in [A1,A2,B,C,D,E,F]]
  names = 'r2_train r2_test mae mse msle rmse rmsle'.split()
  scores = pd.Series(data,index=names)
  return scores

In [72]:
# 다양한 모델을 만들고 성능을 출력하는 함수 작성
# 결과를 DataFrame으로 만듭니다

def make_models(xtrain,xtest,ytrain,ytest,n=300,RL=False):
  temp = pd.DataFrame()

  model1 = LinearRegression().fit(xtrain,ytrain)
  temp['model1'] = get_scores(model1,xtrain,xtest,ytrain,ytest)

  if not RL:
    model2 = DecisionTreeRegressor(random_state=0).fit(xtrain,ytrain)
    temp['model2'] = get_scores(model2,xtrain,xtest,ytrain,ytest)

    for d in range(3,9):
      model2 = DecisionTreeRegressor(max_depth=d,random_state=0).fit(xtrain,ytrain)
      temp[f'model2_{d}'] = get_scores(model2,xtrain,xtest,ytrain,ytest)

    model3 = RandomForestRegressor(n,random_state=0).fit(xtrain,ytrain)
    temp['model3'] = get_scores(model3,xtrain,xtest,ytrain,ytest)

    for d in range(3,9):
      model3= RandomForestRegressor(n,max_depth=d,random_state=0).fit(xtrain,ytrain)
      temp[f'model3_{d}'] = get_scores(model3,xtrain,xtest,ytrain,ytest)

  model4 = XGBRegressor(objective = 'reg:squarederror').fit(xtrain,ytrain)
  temp['model4'] = get_scores(model4,xtrain,xtest,ytrain,ytest)

  if RL:
    for a in [0.01, 0.1, 1, 2]:
      model5 = Ridge(alpha=a).fit(xtrain,ytrain)
      temp['model5'] = get_scores(model5,xtrain,xtest,ytrain,ytest)

    for a in [0.01, 0.1, 1, 2]:
      model6 = Lasso(alpha=a).fit(xtrain,ytrain)
      temp['model6'] = get_scores(model6,xtrain,xtest,ytrain,ytest)

  temp = temp.T
  temp.insert(2,'diff',(temp['r2_train']-temp['r2_test']).abs())

  return temp


### 5-4. 데이터 분리, 모델적용

In [73]:
# [7] X의 최종 전처리 버전인 dfX의 컬럼 확인
dfX.columns

Index(['ID', 'age', 'sex', 'bmi', 'children', 'smoker', 'region'], dtype='object')

In [74]:
# [8] dfX 에서 사용할 feature 선택하여 dfX2로 저장
dfX2 = dfX[['age', 'sex', 'bmi', 'children', 'smoker', 'region']]
dfX2

,age,sex,bmi,children,smoker,region
0,46,0,19.8550,0,0,0
1,25,0,25.8400,1,0,1
2,29,0,27.2000,0,0,2
3,31,0,38.3900,2,0,3
4,57,0,40.2800,0,0,1
...,...,...,...,...,...,...
530,56,0,34.4300,0,0,3
531,21,0,26.0300,0,0,1
532,35,0,39.7100,4,0,1
533,19,0,35.4000,0,0,2


In [75]:
# [9] dfX2 데이터를 XF, X_submissionF 로 다시 분리
# dfX2 분리에서는 X_use의 행의 개수 사용
# Y에서 'charges' 만 YF로 사용
train_size = len(X_use)
XF = dfX2[:train_size]
X_submissionF = dfX2[train_size:]
print(len(XF),len(X_submission))

YF = Y['charges']
Y_submissionF = Y_hidden['charges']
print(len(YF),len(Y_submissionF))


803 535
803 535


In [76]:
# [10] XF, YF 데이터를 사용하여 데이터를 분리하고 make_models를 호출합니다.
xtrain,xtest,ytrain,ytest = train_test_split(XF,YF,
                                             test_size=0.3,
                                             random_state=1234)
models = make_models(xtrain,xtest,ytrain,ytest)
print(models)

          r2_train  r2_test   diff       mae           mse   msle      rmse  \
model1      0.7276   0.8053 0.0777 4088.1861 34101788.3022 2.2322 5839.6736   
model2      1.0000   0.7819 0.2181 2868.9598 38205213.0025 0.1835 6181.0366   
model2_3    0.8585   0.8858 0.0273 2696.0927 19997881.3735 0.1821 4471.8991   
model2_4    0.8727   0.8958 0.0231 2577.9273 18253288.2026 0.1451 4272.3867   
model2_5    0.8874   0.8728 0.0146 2596.6833 22286045.4088 0.1322 4720.8098   
model2_6    0.9099   0.8579 0.0520 2577.6188 24890079.4374 0.1345 4988.9958   
model2_7    0.9336   0.8346 0.0990 2798.3615 28976483.8421 0.1562 5382.9809   
model2_8    0.9529   0.8257 0.1272 2790.4426 30532221.1069 0.1581 5525.5969   
model3      0.9745   0.8859 0.0886 2463.1906 19989477.3933 0.1441 4470.9593   
model3_3    0.8655   0.8890 0.0235 2595.3486 19453820.2124 0.1677 4410.6485   
model3_4    0.8802   0.8934 0.0132 2530.0964 18675883.3582 0.1363 4321.5603   
model3_5    0.8999   0.8920 0.0079 2456.3668 1892692

In [77]:
# [11] models를 평가기준으로 정렬하여 좋은 모델을 알아봄
print(models.sort_values('rmsle').head(5))

          r2_train  r2_test   diff       mae           mse   msle      rmse  \
model3_6    0.9229   0.8915 0.0314 2412.9721 19010932.4231 0.1275 4360.1528   
model3_5    0.8999   0.8920 0.0079 2456.3668 18926929.1843 0.1285 4350.5091   
model3_7    0.9443   0.8900 0.0543 2415.4275 19264118.1168 0.1312 4389.0908   
model2_5    0.8874   0.8728 0.0146 2596.6833 22286045.4088 0.1322 4720.8098   
model2_6    0.9099   0.8579 0.0520 2577.6188 24890079.4374 0.1345 4988.9958   

          rmsle  
model3_6 0.3570  
model3_5 0.3584  
model3_7 0.3622  
model2_5 0.3636  
model2_6 0.3668  


### 5-5. 모델 선택, 결과 제출

In [78]:
# [12] 모델 학습시켜 model 로 저장, submission 만들기, 저장하기
model = RandomForestRegressor(300,max_depth=5,random_state=0).fit(xtrain,ytrain)
print(get_scores(model,xtrain,xtest,ytrain,ytest))

pred = model.predict(X_submissionF)
submission = pd.DataFrame({'ID': X_submission['ID'],
                           'charges': pred})
submission.to_csv('00100.csv',index=False)

r2_train          0.8999
r2_test           0.8920
mae            2456.3668
mse        18926929.1843
msle              0.1285
rmse           4350.5091
rmsle             0.3584
dtype: float64


In [79]:
# [13] Y_submissionF 를 사용하여 성능을 알아봄 (필수 아님)
temp = pd.DataFrame({'ID':X_submission['ID'],
              'charges_pred' : pred,
              'charges_true' : Y_submissionF})
temp.head(10)

,ID,charges_pred,charges_true
0,804,3305.5344,1163.4627
1,805,39382.0442,38792.6856
2,806,8380.3883,18838.7037
3,807,10700.0797,8062.7640
4,808,14171.3555,12979.3580
5,809,13555.1580,30063.5805
6,810,5042.0867,3577.9990
7,811,7279.0083,7727.2532
8,812,4417.3051,2352.9684
9,813,17193.4562,13063.8830


### 5-6.시험보러 가서는 이렇게

In [80]:
# 실제 작업

# import 를 쭈우욱!! 작성
import pandas as pd
# 시험환경에서는 아래와 같이 해야해서 수정했습니다 ^^*
pd.options.display.max_rows = 500     #출력할 max row를 지정
pd.options.display.max_columns = 20   #출력할 max columns를 지정
pd.set_option('display.float_format', '{:.4f}'.format)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_squared_log_error as msle
import numpy as np  # 추가해야함

# 함수들은 만들어야 합니다.

def get_scores_f(model, xtrain, xtest, ytrain, ytest):
    pred1 = model.predict(xtrain)   # train
    pred2 = model.predict(xtest)    # test

    # 음수가 있어도 상관없음
    A1 = r2_score(ytrain, pred1)
    A2 = r2_score(ytest, pred2)
    # msle, rmsle 는 음수가 있어서는 안됨
    pred2 = np.where(pred2<0, 0, pred2)
    B = msle(ytest, pred2)
    C = np.sqrt(B)   # rmsle
    data = [round(x, 4) for x in [A1, A2, C]]
    names = 'r2_train r2_test rmsle'.split()
    scores = pd.Series(data, index=names)
    return  scores

def make_models_f(xtrain, xtest, ytrain, ytest, n=300):
    temp = pd.DataFrame()

    model1 = LinearRegression().fit(xtrain, ytrain)
    temp['model1'] = get_scores_f(model1, xtrain, xtest, ytrain, ytest)

    model2 = DecisionTreeRegressor(random_state=0).fit(xtrain, ytrain)  # overfitting 나기 쉬움
    temp['model2'] = get_scores_f(model2, xtrain, xtest, ytrain, ytest)

    for d in range(3, 9):
        model2 = DecisionTreeRegressor(max_depth=d, random_state=0).fit(xtrain, ytrain)
        temp[f'model2_{d}'] = get_scores_f(model2, xtrain, xtest, ytrain, ytest)

    model3 = RandomForestRegressor(n, random_state=0).fit(xtrain, ytrain)
    temp['model3'] = get_scores_f(model3, xtrain, xtest, ytrain, ytest)

    for d in range(3, 9):
        model3 = RandomForestRegressor(n, max_depth=d, random_state=0).fit(xtrain, ytrain)
        temp[f'model3_{d}'] = get_scores_f(model3, xtrain, xtest, ytrain, ytest)

    model4 = XGBRegressor(objective='reg:squarederror').fit(xtrain, ytrain)
    temp['model4'] = get_scores_f(model4, xtrain, xtest, ytrain, ytest)

    temp = temp.T
    temp.insert(2, 'diff', (temp['r2_train'] - temp['r2_test']).abs())

    return temp

X_use = pd.read_csv('x_train.csv')
X_submission = pd.read_csv('x_test.csv')
Y = pd.read_csv('y_train.csv')
#print([x.shape for x in [X_use, X_submission, Y]]) # [(803, 7), (535, 7), (803, 2)]
# ----------------------------------------------------------------------------------
dfX = pd.concat([X_use, X_submission], ignore_index=True)
#dfX.info()
#print(dfX.nunique())
#print(dfX['region'].unique())

for x in ['sex', 'smoker', 'region']:
    temp = dfX[x].unique()
    dfX[x] = dfX[x].replace(temp, range(len(temp)))
#dfX.info()

#print(dfX.columns)
dfX2 = dfX.drop(columns='ID')
#dfX2.columns

train_size = len(X_use)
XF = dfX2[:train_size]  #  dfX2가 DataFrame, ndarry 인 경우도 행번호로 indexing 가능
X_submissionF = dfX2[train_size:]
YF = Y['charges']
#print([x.shape for x in [XF, YF, X_submissionF]])

xtrain, xtest, ytrain, ytest = train_test_split(XF, YF,
                                                test_size=0.3,
                                                random_state=1234)


#models = make_models_f(xtrain, xtest, ytrain, ytest)
#print(models.sort_values('rmsle').head(5))
model = RandomForestRegressor(300, max_depth=5, random_state=0).fit(xtrain, ytrain)
print(get_scores_f(model, xtrain, xtest, ytrain, ytest))
pred = model.predict(X_submissionF)
submission = pd.DataFrame({'ID': X_submission['ID'],
                           'charges' : pred})
submission.to_csv('0001000.csv', index=False)

<ipython-input-80-21ee064f2585>:79: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfX[x] = dfX[x].replace(temp, range(len(temp)))


r2_train   0.8999
r2_test    0.8920
rmsle      0.3584
dtype: float64


In [81]:
# model3_5    0.8999   0.8920 0.0079 0.3584